This forecast model uses the input table created with the "input transformer code" applied in another Jupyter notebook. This notebook is about monthly forecasting of sales data. For training and model creation; I used most of the historical data as training data and selected the last N months as test set; N is specified at the beginning of code as a parameter.

Test set is consisting of 0 values for test months at beginning. Think about N rows to be predicted. When 1st month of the test set is predicted, it is also written on another rows in a shifting pattern, according to "Last M months" parameter set at the beginning. 

After I get the predictions of model, I compared them to the real life predictions made by sales department for these last N months. 

Model structure is based on a classification stage at first. Dataset is a "0 dense dataset" where about 50% of the data contains 0 as a sales output. This creates some issues for a regression problem. 0 values leverages the predictions towards 0. 

When I divided the problem into 2 stages; I also could neglect 0 values from regression problem. In first stage, classification algorithm decides if a values is 0 or "not 0". If it predicts the value as 0, then a regression algorithm will not be applied for that row and prediction is written a 0. But if it is labeled as "not 0", second stage regression algorithm will be applied for that row and regression result will be written as a prediction.

Classification dataset consists of all the input data (if it is balanced - daily prediction dataset was imbalanced and I applied (80% 0 20% else) an undersampling algorithm to 0 values to make it balanced, I am adding it into this code also as a comment-) whereas regression data neglects 0 values from the input to eliminate their leverage effect. 

Inside this development code there are several parameters for experimental trials like nestedcv parameter, or neural network parameter etc. They are set at the beginning according to the experimental design and model is run according to these settings. Mostly, I am using hyperparameter search on several algorithms to get a better performance from this dataset. And also I use cross validation to get a better estimate of errors from different runs. 

After getting best of the best model among all experiments, a shortened version of this code can be used for further production stages with the best hyperparameters for the selected model, making predictions on newcoming data.

This project is simply for ad-hoc usage. It will be used at a monthly frequency to update monthly forecasts for future sales. 

In [26]:
import pandas as pd
import re
import numpy as np
pd.set_option('display.max_columns',40)
import more_itertools as mi
import timeit
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
import warnings
from sklearn.exceptions import DataConversionWarning, ConvergenceWarning

pd.options.mode.chained_assignment = None

text_file = open(r'C:\Users\ali.kilinc\Desktop\Tahminleme\Output.txt',"w")

warnings.filterwarnings("ignore", category=DataConversionWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

start_time = timeit.default_timer()

forecast = pd.read_csv(r'C:\Users\ali.kilinc\Desktop\Tahminleme\PrepForecastPWCompany10.csv', index_col= False)

param = 12 # last M months parameter to be included as input variable
predmonths = 10 # N months to be predicted in a moving pattern
transform = 1 #If transformation will be applied to the y values or not
pre_classify = 1 #Will pre-classification stage will be applied to the dataset?
regress = 1 # will regression stage will be applied
hypertest = 1 #Hyperparameter search option is on or off 
nestedcv = 0 # nestedcv option is on or off
neural_model = 0 # will a neural model be applied on dataset?
neural_model_final = 0 #If a neural model is trained, after writing the best parameters run a final training on whole dataset
finalize = 0 # if a classical ML model is trained, after writing the best parameters, run a final training on whole dataset

C:\Users\ali.kilinc\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [27]:
forecast['code'] = forecast['code'].astype('str')

#when an xls file converted to csv, leading zeros of the text values are lost This line tries to get them back dynamically
forecast['code'][forecast['code'].str[:1].str.isnumeric() == True]  = forecast['code'][forecast['code'].str[:1].str.isnumeric() == True].str.zfill(10)

print(forecast.info())
print(forecast.describe(include='all'))
print(forecast.columns)

# extracting binary column from a numeric column with a condition
forecast['yclass'] = (forecast['M'] > 0).astype(int)
#forecast['yclass'] = np.where(forecast['M'] > 0, 1, 0)
#forecast['yclass'] = forecast['M'].apply(lambda x: 1 if x > 0 else 0) it is slower than others

forecast['date'] = pd.to_datetime(forecast['date'], format = '%Y-%m') #this one transforms to yyyy mm dd format in default. datetime type cannot be converted into mm - yyyyy
forecast['date'] = forecast['date'].dt.to_period('m')

pred_set = forecast[(forecast['date'] <= forecast['date'].max()) & (forecast['date'] > forecast['date'].max() - (predmonths+1))]
forecast = forecast[forecast['date'] <= forecast['date'].max() - (predmonths+1)]


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65518 entries, 0 to 65517
Data columns (total 41 columns):
code             65518 non-null object
service          65518 non-null object
from             65518 non-null object
to               65518 non-null object
date             65518 non-null object
M-12             65518 non-null int64
M-11             65518 non-null int64
M-10             65518 non-null float64
M-9              65518 non-null float64
M-8              65518 non-null float64
M-7              65518 non-null float64
M-6              65518 non-null float64
M-5              65518 non-null float64
M-4              65518 non-null float64
M-3              65518 non-null float64
M-2              65518 non-null float64
M-1              65518 non-null float64
M                65518 non-null float64
month            65518 non-null int64
quarter          65518 non-null int64
year             65518 non-null int64
quarterind       65518 non-null float64
monthind         65518 non

In [28]:
pred_drop = []

for col in forecast.columns:
    
    if ('M-' in col):
        
        if (int(re.findall('\d+', col)[0]) > param) & (int(re.findall('\d+', col)[0]) > predmonths):
            forecast.drop(col, axis = 1, inplace = True)
            pred_set.drop(col, axis = 1, inplace = True)
            
        elif (int(re.findall('\d+', col)[0]) > param) & (int(re.findall('\d+', col)[0]) <= predmonths):
            forecast.drop(col, axis = 1, inplace = True)
            pred_drop.append(col)
            
        elif (int(re.findall('\d+', col)[0]) <= param) & (int(re.findall('\d+', col)[0]) > predmonths):
            #pred_set.drop(col, axis = 1, inplace = True)
            pred_drop.append(col)
                       
        else:
            pass
    else:
        pass

Above code dynamically sets Past Month input values on training set and test set. Some of them are totally not needed and dropped directly. Some of them are inserted into a list of columns to be dropped while inserting prediction values to test set at the end.

Below code drops some unwanted features from the model. Also plotting part is inactive in this version, they were used for the analysis of data.

In [29]:
forecast.drop(columns= ['date', 'year', 'quarterind', 'monthind'], inplace = True)
print(pred_set.columns)
pred_unique = pred_set['date']
pred_set.drop(columns= ['date', 'year', 'quarterind', 'monthind'], inplace = True)

''' related to first version, but as a library it can stay
for i in range(0,param):
    if i < param - 1:
        pred_set[str('M-'+str(param - i))] = pred_set[str('M-'+str(param - i - 1))]
    else:
        pred_set['M-1'] = pred_set['M']
'''

pred_set.drop(columns= ['M'], inplace = True)

'''
fig, ax = plt.subplots(2,2)

ax[0,0] = forecast['dptholidaycnt'].value_counts().plot(kind = 'bar', ax = ax[0,0])

ax[0,1] = forecast['arvholidaycnt'].value_counts().plot(kind = 'bar', ax = ax[0,1])

ax[1,0] = forecast['dptfridaycnt'].value_counts().plot(kind = 'bar', ax = ax[1,0])

ax[1,1] = forecast['arvfridaycnt'].value_counts().plot(kind = 'bar', ax = ax[1,1])



fig2, ax2 = plt.subplots(1,2)

ax2[0].boxplot(forecast['Dolar'])

ax2[1].boxplot(forecast['Euro'])
'''

Index(['code', 'service', 'from', 'to', 'date', 'M-12', 'M-11', 'M-10', 'M-9',
       'M-8', 'M-7', 'M-6', 'M-5', 'M-4', 'M-3', 'M-2', 'M-1', 'M', 'month',
       'quarter', 'year', 'quarterind', 'monthind', 'moving3', 'moving6',
       'moving12', 'Dolar', 'Euro', 'Pound', 'dptholidaycnt', 'dptmoncnt',
       'dpttuecnt', 'dptwedcnt', 'dptthurscnt', 'dptfridaycnt',
       'arvholidaycnt', 'arvmoncnt', 'arvtuecnt', 'arvwedcnt', 'arvthurscnt',
       'arvfridaycnt', 'yclass'],
      dtype='object')


"\nfig, ax = plt.subplots(2,2)\n\nax[0,0] = forecast['dptholidaycnt'].value_counts().plot(kind = 'bar', ax = ax[0,0])\n\nax[0,1] = forecast['arvholidaycnt'].value_counts().plot(kind = 'bar', ax = ax[0,1])\n\nax[1,0] = forecast['dptfridaycnt'].value_counts().plot(kind = 'bar', ax = ax[1,0])\n\nax[1,1] = forecast['arvfridaycnt'].value_counts().plot(kind = 'bar', ax = ax[1,1])\n\n\n\nfig2, ax2 = plt.subplots(1,2)\n\nax2[0].boxplot(forecast['Dolar'])\n\nax2[1].boxplot(forecast['Euro'])\n"

In [30]:
# some of the columns are concatenated as a different column to get combination information directly
forecast['comb'] = (forecast['code'] + forecast['from'] + forecast['to'])
pred_set['comb'] = (pred_set['code'] + pred_set['from'] + pred_set['to'])

#preparing the moving averages dictionary
movingdict = dict()

for w in [3,6,12]:
    movinglist = []
    for col in pred_set.columns:
        if ('M-' in col):
            if (int(re.findall('\d+', col)[0])) <= w:
                movinglist.append(col)
    movingdict[w] = movinglist

#forecast.drop(columns = ['from' ,'to'], inplace = True)
#pred_set.drop(columns = ['from' ,'to'], inplace = True)


In [31]:
#code for some ad-hoc visualizations. I used them for analysis and deactivated then

'''
#I tried to make a dynamic figure which I can zoom in at the specific areas but I couldn't right now. After saving into computer, it was possible. 
#So i decided to divide all graphs to multiple subplots with a dynamic formula. It was more interpretable. 

figcnt = ceil(len(sorted(set(forecast['comb'])))/35)
plotcnt = 0

for m in range(figcnt):
    fig = str('fig'+'_'+str(figcnt))
    ax = str('ax'+'_'+str(figcnt))
    fig, ax = plt.subplots(7,5)
    plt.subplots_adjust(left = 0.03, right = 0.95, top = 0.95, bottom = 0.05)
    for i ,j in itertools.product(range(7), range(5)):
        if plotcnt < len(sorted(set(forecast['comb']))):
            ax[i,j] = forecast[['comb', 'M']][forecast['comb']==sorted(set(forecast['comb']))[plotcnt]].plot(kind = 'hist', ax = ax[i,j])
            ax[i,j].set(title = sorted(set(forecast['comb']))[plotcnt])
            plotcnt += 1
        else:
            pass

'''
'''
#this code part uses only comb column, transforms a comb name to other if it is above threshold according to 0 values among timespan
# this coluld be done with the code line at table preparation code. This would not delete the line but change the combination at beginning
# same can be applied to from and to seperately. 
forecast.drop(columns = ['from', 'to'], axis=1, inplace = True)

threshold = 0.9

comblist = sorted(set(forecast['comb']))
countlist = []
for item in comblist:
    #subdf = forecast[['comb', str('M-' + str(param)), 'M']][forecast['comb'] == item]
    subdf1 = forecast[['comb', str('M-' + str(param))]][forecast['comb'] == item]
    subdf2 = forecast[['comb', 'M']][forecast['comb'] == item]
    subdf2.columns = ['comb', str('M-' + str(param))]
    #subdf[['comb', str('M-' + str(param))]] = subdf[['comb', str('M-' + str(param))]].append(subdf[['comb','M']].iloc[-param:], ignore_index = True)
    subdf1 = subdf1.append(subdf2.iloc[-param:], ignore_index = True)
    templist = [item, len(subdf1[subdf1[str('M-' + str(param))] == 0])/len(subdf1)]
    countlist.append(templist)
    templist = []

countdf = pd.DataFrame(countlist, columns = ['comb', 'zerocnt'])
#print(list(countdf['comb'][countdf['zerocnt'] > threshold]))

countdf['zerocnt'].plot(kind = 'hist')
#print(countdf['comb'].value_counts())

forecast['comb'].replace(list(countdf['comb'][countdf['zerocnt'] > threshold]) , 'oth', inplace = True)
#print(forecast['comb'].value_counts().head(170))
'''

"\n#this code part uses only comb column, transforms a comb name to other if it is above threshold according to 0 values among timespan\n# this coluld be done with the code line at table preparation code. This would not delete the line but change the combination at beginning\n# same can be applied to from and to seperately. \nforecast.drop(columns = ['from', 'to'], axis=1, inplace = True)\n\nthreshold = 0.9\n\ncomblist = sorted(set(forecast['comb']))\ncountlist = []\nfor item in comblist:\n    #subdf = forecast[['comb', str('M-' + str(param)), 'M']][forecast['comb'] == item]\n    subdf1 = forecast[['comb', str('M-' + str(param))]][forecast['comb'] == item]\n    subdf2 = forecast[['comb', 'M']][forecast['comb'] == item]\n    subdf2.columns = ['comb', str('M-' + str(param))]\n    #subdf[['comb', str('M-' + str(param))]] = subdf[['comb', str('M-' + str(param))]].append(subdf[['comb','M']].iloc[-param:], ignore_index = True)\n    subdf1 = subdf1.append(subdf2.iloc[-param:], ignore_index = 

In [32]:
#seperating input and output variables
df_y = forecast['M']
df_y_class = forecast['yclass']
df_x = forecast.drop(columns = ['M', 'yclass'], axis=1)
pred_set = pred_set.drop(columns = ['yclass'], axis=1)

Variables like quarter, month, weekday are actually cyclic variables besides being categorical variables. Below I made experiments for these variables, either being cyclic or categorical. At the end, model performance was better when they were treated as categorical variables. 

In [33]:
'''
#!!!!!!!! IT IS THE TRANSFORMATION OF CYCLICAL VARIABLES LIKE MONTH AND QUARTER.
df_x['month_x'] = np.sin(2*np.pi*df_x['month']/12)
df_x['month_y'] = np.cos(2*np.pi*df_x['month']/12)

df_x['quarter_x'] = np.sin(2*np.pi*df_x['quarter']/4)
df_x['quarter_y'] = np.sin(2-np.pi*df_x['quarter']/4)

pred_set['month_x'] = np.sin(2*np.pi*pred_set['month']/12)
pred_set['month_y'] = np.cos(2*np.pi*pred_set['month']/12)

pred_set['quarter_x'] = np.sin(2*np.pi*pred_set['quarter']/4)
pred_set['quarter_y'] = np.sin(2-np.pi*pred_set['quarter']/4)

df_x.drop(columns = ['month', 'quarter'], axis = 1, inplace = True)
pred_set.drop(columns = ['month', 'quarter'], axis = 1, inplace = True)
'''

'''
# General function for this transformation
def encode(data, col, max_val): # max val is theoretical maximum of column, for month it is 12, for day of year it is 365 etc.
    data[col + '_sin'] = np.sin(2 * np.pi * data[col]/max_val)
    data[col + '_cos'] = np.cos(2 * np.pi * data[col]/max_val)
    return data
'''

df_x['month'] = df_x['month'].astype('category')
df_x['quarter'] = df_x['quarter'].astype('category')


In [34]:
num_cols = list(df_x._get_numeric_data().columns)

fix_cols = []
for col in df_x.columns:
    if df_x[col].nunique() == 1:
        fix_cols.append(col)
    else:
        pass

emp_cols = []
for col in df_x.columns:
    if df_x[col].count() > 0:
        pass
    else:
        emp_cols.append(col)
        
cat_cols = list(set(df_x.columns) - set(num_cols) - set(emp_cols) - set(fix_cols))
num_cols = list(set(num_cols) - set(fix_cols) - set(emp_cols))

df_x.drop(columns = list(emp_cols+fix_cols), inplace = True)

print(cat_cols)
print(num_cols)

['quarter', 'comb', 'from', 'to', 'month', 'code', 'service']
['dptwedcnt', 'arvfridaycnt', 'dptthurscnt', 'moving12', 'M-11', 'dptfridaycnt', 'moving3', 'Euro', 'arvmoncnt', 'arvwedcnt', 'dpttuecnt', 'M-3', 'M-9', 'M-10', 'M-1', 'Dolar', 'dptmoncnt', 'arvholidaycnt', 'Pound', 'dptholidaycnt', 'moving6', 'M-5', 'M-7', 'M-12', 'arvthurscnt', 'M-2', 'M-8', 'M-6', 'arvtuecnt', 'M-4']


In [35]:
df_x = pd.DataFrame(df_x, columns = df_x.columns)
df_y = pd.DataFrame(df_y, columns = ['M'])
df_y_class = pd.DataFrame(df_y_class, columns = ['yclass'])

#data shuffler
idx = np.random.permutation(df_x.index)
df_x = df_x.reindex(idx)
df_y = df_y.reindex(idx)
df_y_class = df_y_class.reindex(idx)

df_x_regr = df_x.loc[df_y_class[df_y_class['yclass']==1].index]
df_y_regr = df_y.loc[df_y_class[df_y_class['yclass']==1].index]

'''
#On daily dataset, proportion of 0 values to whole dataset is about 75% which means an imbalance on classification dataset
#For algorithm, it is much easier to label most of the data as 0, and this means sacrificing from quality.
#Because of that reason, some of the 0 values are removed from dataset with RandomUnderSampling.
rus = RandomUnderSampler(random_state = 23, sampling_strategy = 1)
arr_x, arr_y_class = rus.fit_resample(df_x.to_numpy(), df_y_class.to_numpy())

df_x = pd.DataFrame(arr_x, columns = df_x.columns)
df_y_class = pd.DataFrame(arr_y_class, columns = ['yclass'])
df_x[['quarter', 'month', 'weekday']] = df_x[['quarter', 'month', 'weekday']].astype('category')
df_x[num_cols] = df_x[num_cols].astype('float64')
'''

"\n#On daily dataset, proportion of 0 values to whole dataset is about 75% which means an imbalance on classification dataset\n#For algorithm, it is much easier to label most of the data as 0, and this means sacrificing from quality.\n#Because of that reason, some of the 0 values are removed from dataset with RandomUnderSampling.\nrus = RandomUnderSampler(random_state = 23, sampling_strategy = 1)\narr_x, arr_y_class = rus.fit_resample(df_x.to_numpy(), df_y_class.to_numpy())\n\ndf_x = pd.DataFrame(arr_x, columns = df_x.columns)\ndf_y_class = pd.DataFrame(arr_y_class, columns = ['yclass'])\ndf_x[['quarter', 'month', 'weekday']] = df_x[['quarter', 'month', 'weekday']].astype('category')\ndf_x[num_cols] = df_x[num_cols].astype('float64')\n"

In [36]:
month_var = []

for i in range(0,param):
    month_var.append(str('M-'+str(param -i)))
    
print(month_var)

['M-12', 'M-11', 'M-10', 'M-9', 'M-8', 'M-7', 'M-6', 'M-5', 'M-4', 'M-3', 'M-2', 'M-1']


In [37]:
#y is a lognormal distribution with mostly 0 values, i will try to fit a log(x+1) transformation for that.( same for all month variables)
#log1p(x) is equal to log(x+1). reverse of it is expm1(y)
#log transformation, or similars, can be applied to whole data as it has no data dependent variable that can lead to a data leakage!!!
#Transform input features if needed, but it is not an assumption of regression(normality of ind variables)
#highly skewed variables can be normalized with transformations to increase predictive power!!!!!
#https://www.researchgate.net/post/Should_I_transform_non-normal_independent_variables_in_logistic_regression/564f3cb65e9d97d2a58b457d/citation/download

if transform == 1:
    #stats.probplot(df_y_regr.values.ravel(),plot=plt)
    #sns.distplot(df_y_regr, hist = False, kde = True)
    #plt.show()
    #df_y_regr = np.log1p(df_y_regr)
    df_y_regr = np.log(df_y_regr)
    #df_y_regr = pow(df_y_regr, 0.5)
    #stats.probplot(df_y_regr.values.ravel(),plot=plt)
    #sns.distplot(df_y_regr, hist = False, kde = True)
    #plt.show()
    #df_x[month_var] = np.log(df_x[month_var]+1)
    #pred_set[month_var] = np.log(pred_set[month_var]+1)
else:
    pass

print(pred_set.columns)
print(df_y_class['yclass'].value_counts())
print(df_y_class.groupby(['yclass']).size())

Index(['code', 'service', 'from', 'to', 'M-12', 'M-11', 'M-10', 'M-9', 'M-8',
       'M-7', 'M-6', 'M-5', 'M-4', 'M-3', 'M-2', 'M-1', 'month', 'quarter',
       'moving3', 'moving6', 'moving12', 'Dolar', 'Euro', 'Pound',
       'dptholidaycnt', 'dptmoncnt', 'dpttuecnt', 'dptwedcnt', 'dptthurscnt',
       'dptfridaycnt', 'arvholidaycnt', 'arvmoncnt', 'arvtuecnt', 'arvwedcnt',
       'arvthurscnt', 'arvfridaycnt', 'comb'],
      dtype='object')
1    27207
0    22977
Name: yclass, dtype: int64
yclass
0    22977
1    27207
dtype: int64


In [38]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder, MinMaxScaler, Normalizer
from sklearn.model_selection import GridSearchCV, StratifiedKFold, KFold, cross_val_score, cross_validate, train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, cohen_kappa_score
from xgboost import XGBClassifier
import sklearn.metrics as m
from lightgbm import LGBMClassifier
import category_encoders as ce

print(sorted(m.SCORERS.keys()))

['accuracy', 'adjusted_mutual_info_score', 'adjusted_rand_score', 'average_precision', 'balanced_accuracy', 'completeness_score', 'explained_variance', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'fowlkes_mallows_score', 'homogeneity_score', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_weighted', 'max_error', 'mutual_info_score', 'neg_brier_score', 'neg_log_loss', 'neg_mean_absolute_error', 'neg_mean_gamma_deviance', 'neg_mean_poisson_deviance', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_median_absolute_error', 'neg_root_mean_squared_error', 'normalized_mutual_info_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'r2', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'roc_auc', 'roc_auc_ovo', 'roc_auc_ovo_weighted', 'roc_auc_ovr', 'roc_auc_ovr_weighted', 'v_measure_score']


In [39]:
if pre_classify == 1:
    
    modelsc = dict()
    
    paramsrfc = {
            'max_depth':[5, 9, 18, 32],
            'n_estimators':[10, 50, 100, 200],
            'min_samples_split': np.linspace(0.1, 1.0, 3, endpoint=True),
            'min_samples_leaf':np.linspace(0.1, 0.5, 3, endpoint=True)
            }
    
    paramslgbc = {
            'max_depth':[5, 9, 18, 32],
            'learning_rate': [0.001, 0.01, 0.1],
            'n_estimators': [50, 100, 200],
            'num_leaves': np.linspace(11,51,3,endpoint = True, dtype = int)
            }
    
    paramsknnc = {'n_neighbors':[6,12,20]}
    
    paramssgdc = {
            'penalty':['l1','l2','elasticnet'],
            'max_iter': np.linspace(100,700,3,endpoint = True),
            'alpha': [0.0001, 0.001, 0.01],
            'n_iter_no_change':np.linspace(5, 17, 3, endpoint=True)
            }
    
    paramsxgbc = {
            'n_estimator': np.linspace(100,500,2,endpoint = True),
            'max_depth': np.linspace(3,15,3, endpoint=True, dtype = int),
            #'min_child_weight':np.linspace(1, 9, 3, endpoint=True),
            #'colsample_bytree': np.linspace(0.3,0.5,3,endpoint =True),
            'learning_rate': [0.001, 0.01, 0.1]
            }
    
    paramslogitc = {
            'C': [0.01, 0.1, 1, 10, 100],
            'penalty': ['l2'],
            'tol': [0.0001, 0.001, 0.01]
            }
    
    #random forest-logit has given "AttributeError, none object has no module named write" error with njobs = -1 for gridsearch and verbose = 1 for Random Forest. I closed verbose
    #modelsc['rf'] = [RandomForestClassifier(), paramsrfc]
    modelsc['lgb'] = [LGBMClassifier(verbosity = 1), paramslgbc]
    ##modelsc['knn'] = [KNeighborsClassifier(), paramsknnc]
    ##modelsc['sgd'] = [SGDClassifier(verbose = 1), paramssgdc]
    #modelsc['gbm'] = [XGBClassifier(seed = 23, verbosity = 1), paramsxgbc]
    #modelsc['logit'] = [LogisticRegression(solver = 'sag'), paramslogitc]

In [56]:
    for key, value in modelsc.items():
        
        start_timec = timeit.default_timer()
                
        scorer = ['roc_auc', 'accuracy', 'f1', 'jaccard', 'precision', 'recall']
                       
        numeric_pipe = make_pipeline(MinMaxScaler(feature_range = (0,1)))
        categoric_pipe = make_pipeline(OneHotEncoder(sparse = True, handle_unknown='ignore'))
        preprocessor = ColumnTransformer(transformers = [('num',numeric_pipe, num_cols), ('cat',categoric_pipe,cat_cols)])
        
        all_pipe = make_pipeline(preprocessor, value[0])
                    
        grid_search = GridSearchCV(all_pipe, value[1], cv=5, verbose=1, refit = 'roc_auc', scoring = scorer, return_train_score = True, n_jobs = -1)
        #scoring option is for defining multiple scorers, otherwise null is OK
        #refit must be chosen if multi scorers is selected. But best_score_, best_params_ etc will give only the result of that metric, cant get multi scores
        #here REFIT option tells you which metric do you want to consider for best_params_ calculation(according to which metric)
     
        grid_search.fit(df_x, df_y_class.values.ravel())
        
        #from the resulting parameters dictionary, we choose the index of best_param_ set. This index will help us the get best param value from subresults
        #metric results are in arraf form, in each array there is a list of results corresponding to the all parameter combinations. 
        ind = grid_search.best_index_      
        
        print("model = {}".format(key), file = text_file)
        print("train_roc = {}, test_roc ={}".format(grid_search.cv_results_['mean_train_roc_auc'][ind], grid_search.cv_results_['mean_test_roc_auc'][ind]), file = text_file)
        print("train_acc = {}, test_acc ={}".format(grid_search.cv_results_['mean_train_accuracy'][ind], grid_search.cv_results_['mean_test_accuracy'][ind]), file = text_file)
        print("train_f1 = {}, test_f1 = {}".format(grid_search.cv_results_['mean_train_f1'][ind], grid_search.cv_results_['mean_test_f1'][ind]), file = text_file)
        print("train_jaccard = {}, test_jaccard = {}".format(grid_search.cv_results_['mean_train_jaccard'][ind], grid_search.cv_results_['mean_test_jaccard'][ind]), file = text_file)
        print("train_precision = {}, test_precision = {}".format(grid_search.cv_results_['mean_train_precision'][ind], grid_search.cv_results_['mean_test_precision'][ind]), file = text_file)
        print("train_recall = {}, test_recall = {}".format(grid_search.cv_results_['mean_train_recall'][ind], grid_search.cv_results_['mean_test_recall'][ind]), file = text_file)
        print("avg_fit_time = {}".format(grid_search.cv_results_['mean_fit_time'][ind]), file = text_file)
        #we are getting best_params_ from grid_search object still, it is valid
        print("best_params = {}".format(grid_search.best_params_), file = text_file)
        print("---%0.1f minutes---" %((timeit.default_timer()-start_timec)/60), file = text_file)
        print("run_start = {}...".format(start_time), file = text_file)
        print("current_time = {}...".format(datetime.now()), file = text_file)
        print("current_time = {}...".format(datetime.now()))
        print("-----------------------------------", file = text_file)

Fitting 5 folds for each of 108 candidates, totalling 540 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  4.8min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed: 10.7min
[Parallel(n_jobs=-1)]: Done 540 out of 540 | elapsed: 13.7min finished


current_time = 2020-01-21 10:10:43.948600...


Above code is a hyperparameter search code with a CV. All included models are run with hyperparameter search and at the end all results are written inside a text file as a summary. Written results are regarding the fold with best score

In [57]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import SGDRegressor, LinearRegression, Lasso, Ridge, ElasticNet, HuberRegressor, Lars, RANSACRegressor, PassiveAggressiveRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.neural_network import MLPRegressor

In [58]:
if regress == 1:

    models = dict()
    
    paramsgbm = {
            'n_estimator': np.linspace(100,500,2,endpoint = True),
            'max_depth': np.linspace(1,9,3, endpoint=True, dtype = int),
            #'min_child_weight':np.linspace(1, 9, 3, endpoint=True),
            'colsample_bytree': np.linspace(0.1,0.9,3,endpoint =True)
            }
    
    paramslr = {
                }
    
    paramslas = {
            'alpha': [0.001, 0.01, 0.1, 1],
            'max_iter': np.linspace(0, 3000, 3, endpoint = True),
            'tol': [0.0001, 0.001, 0.01]
                }
    
    paramsrid = {
            'alpha': [0.001, 0.01, 0.1, 1],
            'max_iter': np.linspace(0, 3000, 3, endpoint = True),
            'tol': [0.0001, 0.001, 0.01]
                }
    
    paramsela = {
            'alpha': [0.001, 0.01, 0.1, 1],
            'max_iter': np.linspace(0, 3000, 3, endpoint = True),
            'tol': [0.0001, 0.001, 0.01],
            'l1_ratio': np.linspace(0.25,0.75, 3, endpoint = True)
            }
    
    paramssgd = {
            'penalty':['none', 'l1','l2','elasticnet'],
            'max_iter': np.linspace(0.1, 3000, 3 ,endpoint = True),
            'alpha': [0.001, 0.01, 0.1, 1],
            'n_iter_no_change':np.linspace(5, 10, 2, endpoint=True),
            'l1_ratio': np.linspace(0.25,0.75,3,endpoint = True)
            }
    
    paramsrf = {
            'max_depth':[#5, 9, 
                         18, 32],
            'n_estimators': [#10, 50, 
                             100, 200],
            'min_samples_split': [0.1, 1.0, 2],
            'min_samples_leaf': [0.1, 0.5, 1]
            }
    
    paramslgb = {
            'max_depth':[5, 9, 18, 32],
            'learning_rate': [0.001, 0.01, 0.1],
            'n_estimators': [10, 50, 100, 200],
            'num_leaves': np.linspace(11,51,3,endpoint = True, dtype = int)
            }
        
    paramslgb2 = {
            'max_depth':[9, 18, 32, 64],
            'learning_rate': [0.001, 0.01, 0.1],
            'n_estimators': [100, 200, 400, 600],
            'num_leaves': np.linspace(11,71,4,endpoint = True, dtype = int)
            }
    
    #models['gbm'] = [XGBRegressor(objective = 'reg:squarederror', booster = 'gbtree', seed = 23, learning_rate = 0.01), paramsgbm]
    #models['lr'] = [LinearRegression(), paramslr]
    #models['las'] = [Lasso(), paramslas]
    #models['rid'] = [Ridge(), paramsrid]
    #models['ela'] = [ElasticNet(), paramsela]
    #models['huber'] = HuberRegressor()
    #models['lars'] = Lars()
    #models['passive'] = PassiveAggressiveRegressor()
    #models['ransac'] = RANSACRegressor()
    #models['sgd'] = [SGDRegressor(), paramssgd]
    #models['rf'] = [RandomForestRegressor(), paramsrf]
    models['lgb'] = [LGBMRegressor(), paramslgb]

Below code tries all active models defined above with their related parameter spaces. 

First part of the if structure is for a single run scheme. 
Second part is for a nestedCV scheme where total run size is outerCV*innerCV*param_space . 
At the last part of if structure, there is a classical gridsearch scheme with a CV. Total run size of this part is CV*param_space .

All results are written inside a txt file afterwards for inspection.

In [61]:
    for key, value in models.items():
        
        if hypertest != 1:
            
            start_time = timeit.default_timer()
            
            numeric_pipe = make_pipeline(MinMaxScaler(feature_range = (-1,1)))
            categoric_pipe = make_pipeline(OneHotEncoder(sparse = True, handle_unknown='ignore'))
            #categoric_pipe = make_pipeline(ce.HashingEncoder())
            preprocessor = ColumnTransformer(transformers = [('num',numeric_pipe, num_cols), ('cat',categoric_pipe,cat_cols)])
            
            all_pipe = make_pipeline(preprocessor, value[0])
            
            cv = KFold(n_splits = 5, random_state = 23, shuffle = True)
            
            scorer = ['neg_mean_squared_error', 'neg_mean_absolute_error', 'r2']
            
            scores = cross_validate(all_pipe, df_x, df_y.values.ravel(), scoring = scorer, cv = cv, return_train_score = True)
            
            print("model = {}".format(key), file = text_file)
            print("train_mse = {}, test_mse ={}".format(np.mean(scores['train_neg_mean_squared_error']), np.mean(scores['test_neg_mean_squared_error'])), file = text_file)
            print("train_mae = {}, test_mae ={}".format(np.mean(scores['train_neg_mean_absolute_error']), np.mean(scores['test_neg_mean_absolute_error'])), file = text_file)
            print("train_r2 = {}, test_r2 = {}".format(np.mean(scores['train_r2']), np.mean(scores['test_r2'])), file = text_file)
            print("avg_fit_time = {}".format(np.mean(scores['fit_time'])), file = text_file)
            print("---%0.1f minutes---" %((timeit.default_timer()-start_time)/60), file = text_file)
            print("...{}...".format(start_time), file = text_file)
            print("current_time = {}...".format(datetime.now()), file = text_file)
            print("current_time = {}...".format(datetime.now()))
            print("-----------------------------------", file = text_file)
        
        else:
            if nestedcv == 1:
                
                start_time = timeit.default_timer()
                 
                numeric_pipe = make_pipeline(MinMaxScaler(feature_range = (-1,1)))
                categoric_pipe = make_pipeline(OneHotEncoder(sparse = True, handle_unknown='ignore'))
                preprocessor = ColumnTransformer(transformers = [('num',numeric_pipe, num_cols), ('cat',categoric_pipe,cat_cols)])
                
                all_pipe = make_pipeline(preprocessor, value[0])
                
                grid_search = GridSearchCV(all_pipe, value[1], cv=3, verbose=0)
                               
                cv = KFold(n_splits = 5, random_state = 23, shuffle = False)
                 
                scorer = ['neg_mean_squared_error', 'neg_mean_absolute_error', 'r2']
                #scores = cross_val_score(all_pipe, df_x, df_y.values.ravel(), scoring = 'neg_mean_squared_error', cv=cv)
                #r2 = cross_val_score(all_pipe, df_x, df_y.values.ravel(), scoring = 'r2', cv=cv)
                scores = cross_validate(grid_search, df_x, df_y.values.ravel(), scoring = scorer, cv = cv, return_train_score = True)
                
                #you have to fit grid_search object in order to get best_params_ from it. But here you also need a pipeline for preprocessing. 
                #fitting on pipeline object will give us the desired score. If it was not a pipeline, we would fit grid_search object after preprocessing
                scores.fit(df_x, df_y.values.ravel())     
                
                print("model = {}".format(key), file = text_file)
                print("train_mse = {}, test_mse ={}".format(np.mean(scores['train_neg_mean_squared_error']), np.mean(scores['test_neg_mean_squared_error'])), file = text_file)
                print("train_mse = {}, test_mse ={}".format(np.mean(scores['train_neg_mean_absolute_error']), np.mean(scores['test_neg_mean_absolute_error'])), file = text_file)
                print("train_r2 = {}, test_r2 = {}".format(np.mean(scores['train_r2']), np.mean(scores['test_r2'])), file = text_file)
                print("avg_fit_time = {}".format(np.mean(scores['fit_time'])), file = text_file)
                #we ar getting best_params_ from grid_search object still, it is valid
                print("best_params = {}".format(grid_search.best_params_), file = text_file)
                print("---%0.1f minutes---" %((timeit.default_timer()-start_time)/60), file = text_file)
                print("...{}...".format(start_time), file = text_file)
                print("current_time = {}...".format(datetime.now()), file = text_file)
                print("current_time = {}...".format(datetime.now()))
                print("-----------------------------------", file = text_file)
                
                best_params = dict()
                best_params[key] = grid_search.best_params_
                
                best_results = dict()
                best_results[key] = [np.mean(scores['test_neg_mean_squared_error']), np.mean(scores['test_neg_mean_absolute_error']), np.mean(scores['test_r2'])]
            
            #this part is for only gridsearchcv. We can use CV results, as well as best parameters of the grid search from here. 
            else:
                
                start_time = timeit.default_timer()
                
                scorer = ['neg_mean_squared_error', 'neg_mean_absolute_error', 'r2']
                
                numeric_pipe = make_pipeline(MinMaxScaler(feature_range = (0,1)))
                categoric_pipe = make_pipeline(OneHotEncoder(sparse = True, handle_unknown='ignore'))
                preprocessor = ColumnTransformer(transformers = [('num',numeric_pipe, num_cols), ('cat',categoric_pipe,cat_cols)])
                
                all_pipe = make_pipeline(preprocessor, value[0])
                
                #njobs = -1 uses 2 cores (all available) and 4 threads; consumes 100% of CPU. But it is faster absolutely. Njobs = 1 uses 1 core only, slower but consumes less CPU
                grid_search = GridSearchCV(all_pipe, value[1], cv=5, verbose=1, refit = 'neg_mean_squared_error', scoring = scorer, return_train_score = True, n_jobs = -1)
                #scoring option is for defining multiple scorers, otherwise null is OK
                #refit must be chosen if multi scorers is selected. But best_score_, best_params_ etc will give only the result of that metric, cant get multi scores
                #here REFIT option tells you which metric do you want to consider for best_params_ calculation(according to which metric)
                                
                grid_search.fit(df_x_regr, df_y_regr.values.ravel())
                #all_pipe.fit(df_x, df_y.values.ravel())
                
                #from the resulting parameters dictionary, we choose the index of best_param_ set. This index will help us the get best param value from subresults
                #metric results are in arraf form, in each array there is a list of results corresponding to the all parameter combinations. 
                ind = grid_search.best_index_      
                
                print("model = {}".format(key), file = text_file)
                print("train_mse = {}, test_mse ={}".format(grid_search.cv_results_['mean_train_neg_mean_squared_error'][ind], grid_search.cv_results_['mean_test_neg_mean_squared_error'][ind]), file = text_file)
                print("train_mae = {}, test_mae ={}".format(grid_search.cv_results_['mean_train_neg_mean_absolute_error'][ind], grid_search.cv_results_['mean_test_neg_mean_absolute_error'][ind]), file = text_file)
                print("train_r2 = {}, test_r2 = {}".format(grid_search.cv_results_['mean_train_r2'][ind], grid_search.cv_results_['mean_test_r2'][ind]), file = text_file)
                print("avg_fit_time = {}".format(grid_search.cv_results_['mean_fit_time'][ind]), file = text_file)
                #we are getting best_params_ from grid_search object still, it is valid
                print("best_params = {}".format(grid_search.best_params_), file = text_file)
                print("---%0.1f minutes---" %((timeit.default_timer()-start_time)/60), file = text_file)
                print("...{}...".format(start_time), file = text_file)
                print("current_time = {}...".format(datetime.now()), file = text_file)
                print("current_time = {}...".format(datetime.now()))
                print("-----------------------------------", file = text_file)
                
                best_params = dict()
                best_params[key] = grid_search.best_params_
                
                best_results = dict()
                best_results[key] = [grid_search.cv_results_['mean_test_neg_mean_squared_error'][ind], grid_search.cv_results_['mean_test_neg_mean_absolute_error'][ind], grid_search.cv_results_['mean_test_r2'][ind]]

Fitting 5 folds for each of 144 candidates, totalling 720 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   24.3s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 720 out of 720 | elapsed:  6.8min finished


current_time = 2020-01-21 10:31:35.788602...


In [62]:
if neural_model == 1:
    
    from keras.models import Sequential
    from keras.layers import Dense
    from keras.wrappers.scikit_learn import KerasRegressor
    from keras.optimizers import RMSprop, SGD
    from keras.callbacks import EarlyStopping, ModelCheckpoint
    from keras.models import load_model
    
    start_time = timeit.default_timer()
    
    #these are inactive if cross validation will be used. Test set is applied several times for CV.
    #x_train, x_test, y_train, y_test = train_test_split(df_x, df_y.values.ravel(), test_size = 0.1, random_state = 23)
    #x_train, x_test, y_train, y_test = train_test_split(df_x, df_y.values.ravel(), test_size = 0.25)
    
    numeric_pipe = make_pipeline(Normalizer())
    #numeric_pipe = make_pipeline(MinMaxScaler(feature_range = (-1,1)))
    #SPARSE MATRICES MAY BE MORE EFFICIENT BUT IT IS NOT VERY SUITABLE FOR KERAS MODELS
    categoric_pipe = make_pipeline(OneHotEncoder(sparse = False, handle_unknown='ignore'))
    preprocessor = ColumnTransformer(transformers = [('num',numeric_pipe, num_cols), ('cat',categoric_pipe,cat_cols)])
            
    all_pipe2 = make_pipeline(preprocessor)

Above code is related to the neural network training. Preprocess part starts here. Then there is an inactive single run part below(Cross validation for neural networks is mostly ineffective because of huge datasets but in this case, my dataset is not huge and I could run a CV model easily) and also an active CV run part. NORMALIZATON, and ALSO SCALING AFTERWARDS, IS CRUCIAL FOR NNet preprocessing.!!!

In [63]:
    '''   
    x_train = all_pipe2.fit_transform(x_train)
    x_test = all_pipe2.transform(x_test)
    
    #target_pipe = make_pipeline(Normalizer())
    
    #y_train = target_pipe.fit_transform(y_train.reshape(-1,1))
    #y_test = target_pipe.transform(y_test.reshape(-1,1))
    #reshape is required for normalization like transformations of 1D arrays(output values for example)
    
    optimizer = RMSprop(0.001)
    sgd = SGD(lr=0.1, momentum=0.8,nesterov=False)
      
    model = Sequential()
    model.add(Dense(30, input_dim=x_train.shape[1], kernel_initializer='normal', activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='linear'))
    model.summary()
    
    model.compile(loss='mse', optimizer='adam', metrics=['mse','mae'])
    
    patience = 100
    #early stopping stops training after n unsuccesful runs. Model checkpoint saves that best model n epochs before termination.
    callbacks = [EarlyStopping(monitor='val_mean_squared_error', patience=patience),
                 ModelCheckpoint(filepath='best_model.h5', monitor='val_mean_squared_error', save_best_only=True)]
    #callbacks = [EarlyStopping(monitor='val_mean_squared_error', patience=patience)]   
    
    history = model.fit(x_train, y_train, epochs=10000, batch_size=30,  verbose=1, validation_split=0.2, callbacks = callbacks)
    
    #scores = model.evaluate(x_test, y_test, verbose=0)
    #print(scores)
    #print(model.metrics_names)

   
    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Mean_Square_Error')
    plt.gca().set_xlim([100, np.max(history.epoch)])
    plt.plot(history.epoch, history.history['mean_squared_error'],label='Train Error')
    plt.plot(history.epoch, history.history['val_mean_squared_error'],label = 'Val Error')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()
    
    print("Average of last +-25 best epochs of best epoch:")
    print(np.mean(history.history['val_mean_squared_error'][(np.max(history.epoch)-patience-25):(np.max(history.epoch)-patience+25)]))
    print("............................")
    
    saved_model = load_model('best_model.h5')
    
    loss, mse, mae = saved_model.evaluate(x_test, y_test, verbose=0)
    print("Test set MSE: {}".format(mse))
    print("Test set MAE: {}".format(mae))
    print("Test set LOSS: {}".format(loss))
    print("---%0.1f minutes---" %((timeit.default_timer()-start_time)/60))
    
    ### IF you use a CROSS VALIDATION and your results vary so much, this means you need to increase epoch, increase learning rate
    # or decrease batch size, or all. 
    '''

'   \nx_train = all_pipe2.fit_transform(x_train)\nx_test = all_pipe2.transform(x_test)\n\n#target_pipe = make_pipeline(Normalizer())\n\n#y_train = target_pipe.fit_transform(y_train.reshape(-1,1))\n#y_test = target_pipe.transform(y_test.reshape(-1,1))\n#reshape is required for normalization like transformations of 1D arrays(output values for example)\n\noptimizer = RMSprop(0.001)\nsgd = SGD(lr=0.1, momentum=0.8,nesterov=False)\n  \nmodel = Sequential()\nmodel.add(Dense(30, input_dim=x_train.shape[1], kernel_initializer=\'normal\', activation=\'relu\'))\nmodel.add(Dense(10, activation=\'relu\'))\nmodel.add(Dense(1, activation=\'linear\'))\nmodel.summary()\n\nmodel.compile(loss=\'mse\', optimizer=\'adam\', metrics=[\'mse\',\'mae\'])\n\npatience = 100\n#early stopping stops training after n unsuccesful runs. Model checkpoint saves that best model n epochs before termination.\ncallbacks = [EarlyStopping(monitor=\'val_mean_squared_error\', patience=patience),\n             ModelCheckpoint(fi

In [ ]:
    kfold = KFold(n_splits = 5, random_state = 23, shuffle = True)
    
    epochbestlist = []
    epochbestvallist = []
    epochavglist = []
    epochvalavglist = []
    testmselist = []
    testmaelist = []
    testlosslist= []
    
    ### IF you use a CROSS VALIDATION and your results vary so much, this means you need to increase epoch, increase learning rate
    # or decrease batch size, or all. 
    for train_id, test_id in kfold.split(df_x_regr, df_y_regr.values.ravel()):
    
        patience = 50

        cv_x = df_x_regr.iloc[train_id]
        cv_x_test = df_x_regr.iloc[test_id]
        #normally, if we pass column transformer pipeline into CROSS_VAL_SCORE directly etc, we also define DATAFRAMES with it. but here if we use df_x.values[train_id]
        #it is not a dataframe anymore. and we cant pass it into a column transformer pipeline like that. So, I used a datafram filter not to lose the structure. 
        
        cv_y = df_y_regr.values.ravel()[train_id]
        cv_y_test = df_y_regr.values.ravel()[test_id]
        
        print(cv_x)
        print(cv_y)
        print(cv_x_test)
        
        optimizerr = RMSprop(0.001)
        
        cv_x_pre = all_pipe2.fit_transform(cv_x)
        cv_x_pre_test = all_pipe2.transform(cv_x_test)
        
        model = Sequential()
        model.add(Dense(100, input_dim=cv_x_pre.shape[1], kernel_initializer='normal', activation='relu'))
        model.add(Dense(100, activation='relu'))
        model.add(Dense(50, activation='relu'))
        model.add(Dense(1, activation='linear'))
        model.summary()
        model.compile(loss='mse', optimizer=optimizerr, metrics=['mse','mae'])  
        
        callbacks = [EarlyStopping(monitor='val_mean_squared_error', patience=patience),
                 ModelCheckpoint(filepath='best_model.h5', monitor='val_mean_squared_error', save_best_only=True)]  
        
        history = model.fit(cv_x_pre, cv_y, callbacks = callbacks, epochs = 10000, batch_size = 10, verbose = 1, validation_split = 0.20)
        
        epochbest = np.min(history.history['mean_squared_error'])
        epochvalbest = np.min(history.history['val_mean_squared_error'])
        
        epochavg = np.mean(history.history['mean_squared_error'][(np.max(history.epoch)-patience-5):(np.max(history.epoch)-patience+5)])
        epochvalavg = np.mean(history.history['val_mean_squared_error'][(np.max(history.epoch)-patience-5):(np.max(history.epoch)-patience+5)])
        
        saved_model = load_model('best_model.h5')
        
        loss, mse, mae = saved_model.evaluate(cv_x_pre_test, cv_y_test, verbose=0)
        
        print("Best epoch train MSE:", file = text_file)
        print(epochbest, file = text_file)
        print("............................", file = text_file)
        
        print("Best epoch validation MSE:", file = text_file)
        print(epochvalbest, file = text_file)
        print("............................", file = text_file)
        
        print("Average MSE of last +-25 best epochs of best epoch (train):", file = text_file)
        print(epochavg, file = text_file)
        print("............................", file = text_file)
        
        print("Average MSE of last +-25 best epochs of best epoch (validation):", file = text_file)
        print(epochvalavg,file = text_file)
        print("............................", file = text_file)

        print("Test set MSE: {}".format(mse), file = text_file)
        print("Test set MAE: {}".format(mae), file = text_file)
        print("Test set LOSS: {}".format(loss), file = text_file)
                
        epochbestlist.append(epochbest)
        epochbestvallist.append(epochvalbest)
        epochavglist.append(epochavg)
        epochvalavglist.append(epochvalavg)
        testmselist.append(mse)
        testmaelist.append(mae)
        testlosslist.append(loss)
        
        
    print('Mean of epoch best MSE:',np.mean(epochbestlist), file = text_file)
    print('Mean of epoch best validation MSE:',np.mean(epochbestvallist), file = text_file)
    print('Mean of epoch avg MSE:',np.mean(epochavglist), file = text_file)
    print('Mean of epoch avg validation MSE:',np.mean(epochvalavglist), file = text_file)
    print('Mean test MSE:',np.mean(testmselist), file = text_file)
    print('Mean test MAE:',np.mean(testmaelist), file = text_file)
    print('Mean test LOSS:',np.mean(testlosslist), file = text_file)
    print("current_time = {}...".format(datetime.now()), file = text_file)
    print("current_time = {}...".format(datetime.now()))
    print("---%0.1f minutes---" %((timeit.default_timer()-start_time)/60), file = text_file)

In the neural model above with CV, each fold score is stored inside lists. Inside each fold, a patience value is determined and fold automatically stops after P not-improved epochs from best epoch. And the best epoch scores are stored for this fold. 

After each folds are completed, mean of these best scores are summarized at the end of process. 

Below code is about finalizing the best model with full dataset and making predictions on it.

After choosing the best hyperparameters of the best model at previous section, these hyperparameters are written into this part. Then we are applying a preprocessing part, again, on whole data. Due to the caution of data leakage preprocessing is applied on train data only. In here, train data is whole data. So we have to preprocess them together. 

As it is a 2 staged model, first one is a classification model and the next one is a regression model, and there are different input datasets for each one, I constructed 2 different pipelines for each preprocessing section. 

Then code iterates over each company_code, from, to combination. It retrieves data from prediction dataset for a combination C. It gets predictions rowwise. 1st row is applied on classification first, if it is not 0, then it is applied on regression algorithm and gets a prediction. This prediction value (after a reverse transformation is applied, if it is transformed before) is written into available spots for next month values of other rows. (For example it is written into M-1 column for next month's row and it is also written into M-2 column for (i+2)th row etc. With the help of this approach, all empty places inside future months are filled with prediction values. 

Each combination inside prediction dataset is predicted like this, then they will appear as a single row at the end with predicted values for next N months appearing as columns inside output. 


In [66]:
if finalize == 1:
    
    #rfc = RandomForestClassifier(max_depth = 14.28, min_samples_leaf = 0.1, min_samples_split = 0.1, n_estimators = 200)
    lgbc = LGBMClassifier(learning_rate = 0.1, max_depth = 32, n_estimators = 200, num_leaves = 51)
    #sgdc = SGDClassifier(alpha = 0.0001, max_iter = 100, n_iter_no_change = 11, penalty = 'elasticnet')
    
    lgbr = LGBMRegressor(learning_rate= 0.1, max_depth= 9, n_estimators= 200, num_leaves= 31)

    numeric_pipe = make_pipeline(MinMaxScaler(feature_range = (0,1)))
    categoric_pipe = make_pipeline(OneHotEncoder(sparse = True, handle_unknown='ignore'))
    #categoric_pipe = make_pipeline(ce.HashingEncoder())
    preprocessor = ColumnTransformer(transformers = [('num',numeric_pipe, num_cols), ('cat',categoric_pipe,cat_cols)])
    
    regr_pipe_final = make_pipeline(preprocessor, lgbr)
    
    regr_pipe_final.fit(df_x_regr, df_y_regr.values.ravel())
    #regr_pipe_final.fit(df_x, df_y.values.ravel())
    
    class_pipe_final = make_pipeline(preprocessor, lgbc)
    
    class_pipe_final.fit(df_x, df_y_class.values.ravel())
    
    #unique_everseen function gets a set from list, removing duplicates, but it also obeys the ordering. SET function do not take orders into account.
    print(list(mi.unique_everseen(pred_set['code'] + pred_set['from'] + pred_set['to'])))
    
    pred_final = pd.DataFrame()
    prob_list = []
    for key in list(mi.unique_everseen(pred_set['code'] + pred_set['from'] + pred_set['to'])):
    #for key in ['0000124707DETR']:
    #for key in list(mi.unique_everseen(pred_set['code'] + pred_set['comb'])):
        
        pred_val_list = []
        
        pred_subset = pred_set[(pred_set['code'] + pred_set['from'] + pred_set['to']) == key]
        #pred_subset = pred_set[(pred_set['code'] + pred_set['comb']) == key]
        
        lag = 0          
        #print(pred_subset)
        for i in range(0,predmonths):
                    
            #iloc[i,:] returns series with columns as indexes for single row output but this returns dataframe with proper indexing
            pred = pred_subset.iloc[[i],:]
            #print(pred)
            #moving averages should be calculated dynamically before making a prediction for a row
            for key2, value in movingdict.items():               
                pred.loc[:,'moving'+str(key2)] = (pred[value].sum(axis=1))/key2
            #print(pred)
            pred.drop(columns = pred_drop, axis = 1, inplace = True)
            #print(pred)
            class_val = class_pipe_final.predict(pred)
            
            class_prob = class_pipe_final.predict_proba(pred)
            #print(class_val)
            regr_val = regr_pipe_final.predict(pred)
            #print(regr_val)
            if class_val[0] == 0:
                prob_list.append(class_prob[0][0])
            else:
                pass
            
            for j in range (i+1, predmonths+1):
                #using directly this one do not WORK. FILTERED dataframe is assumed as COPY, and it is not owerwritten on it, i guess. I couldn't find
                # I was directly trying to replace on pred_set, but it didn't work. I created pred_subset and pred_final eventually.
                #print(pred_set.loc[(pred_set['from'] + pred_set['to']) == key, :].iloc[j, pred_set.columns.get_loc(str('M-'+str(j)))])
                #it may still raise the same error but it is a kind of bug and, if you used iloc do not think about that. It is written on documentation also
                
                if (class_val[0] == 0):
                #if (class_prob[0][0] >= 0.40):
                    
                    pred_subset.iloc[j, pred_subset.columns.get_loc(str('M-'+str(j))) + lag] = class_val[0]
                    
                else:
                    if transform == 1:
                        pred_subset.iloc[j, pred_subset.columns.get_loc(str('M-'+str(j))) + lag] = np.expm1(regr_val[0])
                    else:
                        pred_subset.iloc[j, pred_subset.columns.get_loc(str('M-'+str(j))) + lag] = regr_val[0]
            lag = lag + 1
        #print(pred_subset)
        
        #last moving average calculation for a combination, which will appear on final output.
        for key3, value in movingdict.items():               
            pred_subset.loc[:,'moving'+str(key3)] = (pred_subset[value].sum(axis=1))/key3
            
        #print(pred_subset)
        pred_final = pred_final.append(pred_subset)

    pred_final = pd.merge(pred_final, pred_unique, left_index=True, right_index = True)
    pred_final = pred_final[pred_final['date'] == pred_final['date'].max()]
        
    #print_excel = pred_final.to_excel(r'C:\Users\ali.kilinc\Desktop\Genel8MHalfrWtNcyc.xlsx', index = None, header = True, sheet_name = 'FullTable')
    #print_excel = pred_final.to_excel(r'C:\Users\ali.kilinc\Desktop\Last12CompWtNCyc10comb.xlsx', index = None, header = True, sheet_name = 'FullTable')
    #print_excel = pred_final.to_excel(r'C:\Users\ali.kilinc\Desktop\FinalForecastClassAddedSpot.xlsx', index = None, header = True, sheet_name = 'FullTable')



Below code is for finalization of Neural Model. If we decided to run a neural model and get the results from that, this part can be activated and best parameters should be written inside proper places before running the code.

It will again run on whole dataset and there are also proper preprocessing pipelines because of that. 

In [67]:
if neural_model_final == 1:
        
    numeric_pipe = make_pipeline(Normalizer())
    #numeric_pipe = make_pipeline(MinMaxScaler(feature_range = (-1,1)))
    categoric_pipe = make_pipeline(OneHotEncoder(sparse = False, handle_unknown='ignore'))
    preprocessor = ColumnTransformer(transformers = [('num',numeric_pipe, num_cols), ('cat',categoric_pipe,cat_cols)])
    all_pipe3 = make_pipeline(preprocessor)
        
    df_x_n = all_pipe3.fit_transform(df_x)
    
    optimizerr = RMSprop(0.001)
    
    model = Sequential()
    model.add(Dense(100, input_dim=df_x_n.shape[1], kernel_initializer='normal', activation='relu'))
    #model.add(Dense(50, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(1, activation='linear'))
    model.summary()
    model.compile(loss='mse', optimizer=optimizerr, metrics=['mse','mae'])  
    
    callbacks = [EarlyStopping(monitor='val_mean_squared_error', patience=patience),
             ModelCheckpoint(filepath='best_model_final.h5', monitor='val_mean_squared_error', save_best_only=True)] 
    
    history = model.fit(df_x_n, df_y, callbacks = callbacks, epochs = 10000, batch_size = 20, verbose = 1, validation_split = 0.2)
    
    saved_model_final = load_model('best_model_final.h5')

    print(pred_set.head(10))
    
    #unique_everseen function gets a set from list, removing duplicates, but it also obeys the ordering. SET function do not take orders into account.
    print(list(mi.unique_everseen(pred_set['code'] + pred_set['from'] + pred_set['to'])))
    
    pred_final = pd.DataFrame()
    
    for key in list(mi.unique_everseen(pred_set['code'] + pred_set['from'] + pred_set['to'])):
        
        pred_val_list = []
        
        pred_subset = pred_set[(pred_set['code'] + pred_set['from'] + pred_set['to']) == key]
        
        lag = 0
            
        for i in range(0,predmonths):
                
            pred = pred_subset.iloc[[i],:]

            for key2, value in movingdict.items():               
                pred.loc[:,'moving'+str(key2)] = (pred[value].sum(axis=1))/key2

            pred.drop(columns = pred_drop, axis = 1, inplace = True)
    
            pred_val = saved_model_final.predict(all_pipe3.transform(pred))
                
            for j in range (i+1, predmonths+1):
                    
                if transform == 1:
                    pred_subset.iloc[j, pred_subset.columns.get_loc(str('M-'+str(j))) + lag] = np.expm1(pred_val[0])
                else:
                    pred_subset.iloc[j, pred_subset.columns.get_loc(str('M-'+str(j))) + lag] = pred_val[0]
                        
            lag = lag + 1
        
        for key3, value in movingdict.items():               
            pred_subset.loc[:,'moving'+str(key3)] = (pred_subset[value].sum(axis=1))/key3
        
        pred_final = pred_final.append(pred_subset)
       
    pred_final = pd.merge(pred_final, pred_unique, left_index=True, right_index = True)
    
    pred_final = pred_final[pred_final['date'] == pred_final['date'].max()]
        
    print_excel = pred_final.to_excel(r'C:\Users\ali.kilinc\Desktop\Tahminleme\FinalForecastNN4.xlsx', index = None, header = True, sheet_name = 'FullTable')
    

In [68]:
text_file.close()